In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.metrics import AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50, DenseNet121
import tensorflow as tf
import psutil
from pympler import asizeof
import sklearn

In [2]:
csv_file_path = '../DL_for_Hin_Chest_X_Ray/Data_Entry_2017_filtered_2.csv'
csv_file_path = '../DL_for_Hin_Chest_X_Ray/HIN_archive/Data_Entry_2017.csv'
df = pd.read_csv(csv_file_path)

IMAGE_DIR = "../DL_for_Hin_Chest_X_Ray/HIN_archive/images/"
ALL_LABELS = sorted(df["Finding Labels"].str.split("|").explode().unique())
ALL_LABELS_WITHOUT_NO = [l for l in ALL_LABELS if l != "No Finding"]
NUMBER_CLASSES = len(ALL_LABELS)

def preprocess_image(file_path, image_size):
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None
    image = cv2.resize(image, (image_size, image_size))
    return image

def prepare_data(df, image_size, image_dir=IMAGE_DIR):
    images = []
    labels = []
    
    for _, row in tqdm(df.iterrows(), total=len(df)):
        image_path = os.path.join(image_dir, row["Image Index"])
        image = preprocess_image(image_path, image_size)
        
        if image is not None:
            images.append(image)

            current_label = np.zeros(NUMBER_CLASSES, dtype=int)
        
            if row["Finding Labels"] != "No Finding":
                indices = [i for i, label in enumerate(ALL_LABELS_WITHOUT_NO) if label in sorted(row["Finding Labels"].split("|"))]
                for idx in indices:
                    if 0 <= idx < NUMBER_CLASSES:
                        current_label[idx] = 1
            else:
                current_label[NUMBER_CLASSES - 1] = 1
            labels.append(current_label)
    
    images = np.array(images).reshape(-1, image_size, image_size)
    images = np.repeat(images[..., np.newaxis], 3, axis=-1) #rgb
    
    labels = np.array(labels)
    
    return images, labels

def prepare_data_as_paper(df, image_size, image_dir=IMAGE_DIR):
    images = []
    labels = []
    
    for _, row in tqdm(df.iterrows(), total=len(df)):
        image_path = os.path.join(image_dir, row["Image Index"])
        image = preprocess_image(image_path, image_size)
        
        if image is not None:
            images.append(image)

            current_label = np.zeros(NUMBER_CLASSES - 1, dtype=int)
        
            if row["Finding Labels"] != "No Finding":
                indices = [i for i, label in enumerate(ALL_LABELS_WITHOUT_NO) if label in sorted(row["Finding Labels"].split("|"))]
                for idx in indices:
                    if 0 <= idx < NUMBER_CLASSES:
                        current_label[idx] = 1
            labels.append(current_label)
    
    images = np.array(images).reshape(-1, image_size, image_size)
    images = np.repeat(images[..., np.newaxis], 3, axis=-1) #rgb
    
    labels = np.array(labels)
    
    return images, labels

In [3]:
def weighted_cross_entropy(y_true, y_pred):
    p = tf.reduce_sum(y_true)
    n = tf.cast(tf.size(y_true), tf.float32) - p
    beta_p = (p + n) / (p + 1e-7)
    beta_n = (p + n) / (n + 1e-7)
    loss = -beta_p * y_true * tf.math.log(y_pred + 1e-7) - beta_n * (1 - y_true) * tf.math.log(1 - y_pred + 1e-7) # Add epsilon to avoid log(0)
    return tf.reduce_mean(loss)

In [24]:
def weighted_cross_entropy2(y_true, y_pred):
    tf.print("y_true", y_true, summarize=-1)
    tf.print("y_pred", y_pred, summarize=-1)

    positive_cases = tf.equal(y_true[:, -1], 0)  # Positive when last column is 0
    negative_cases = tf.equal(y_true[:, -1], 1)  # Negative when last column is 1
    positive_cases = tf.cast(positive_cases, tf.float32)
    negative_cases = tf.cast(negative_cases, tf.float32)
    
    tf.print("positive_cases", positive_cases, summarize=-1)
    tf.print("negative_cases", negative_cases, summarize=-1)
    
    count_positive = tf.reduce_sum(positive_cases)
    count_negative = tf.reduce_sum(negative_cases)
    
    tf.print("count_positive", count_positive, summarize=-1)
    tf.print("count_negative", count_negative, summarize=-1)
    
    total_samples = tf.cast(tf.shape(y_true)[0], tf.float32)
    
    tf.print("total_samples", total_samples)

    beta_p = total_samples / count_positive
    beta_n = total_samples / count_negative
    
    beta_p = tf.clip_by_value(beta_p, 1e-7, 1e7) #  Clipping 
    beta_n = tf.clip_by_value(beta_n, 1e-7, 1e7)
    
    tf.print("beta_p", beta_p, summarize=-1)
    tf.print("beta_n", beta_n, summarize=-1)
    
    y_pred_binary = tf.stack([tf.reduce_sum(y_pred[:, :-1], axis=-1), y_pred[:, -1]], axis=-1)

    
    y_pred_binary = tf.cast(y_pred_binary, tf.float32)

    tf.print("y_pred_binary", y_pred_binary, summarize=-1)
    
    weighted_positive_loss = beta_p * tf.reduce_sum(-positive_cases * tf.math.log(y_pred_binary[:, 0] + 1e-7))
    weighted_negative_loss = beta_n * tf.reduce_sum(-negative_cases * tf.math.log(y_pred_binary[:, 1] + 1e-7))
    
    tf.print("weighted_positive_loss", weighted_positive_loss, summarize=-1)
    tf.print("weighted_negative_loss", weighted_negative_loss, summarize=-1)   
    
    return weighted_positive_loss + weighted_negative_loss

In [5]:
def create_resnet_model(image_size):
    base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(image_size, image_size, 3), pooling=None)
    base_model.trainable = False
    
    model = Sequential()
    model.add(base_model)
    model.add(Conv2D(2048, (1, 1), activation="relu")) # Transition Layer
    model.add(GlobalAveragePooling2D())   
    model.add(Dense(NUMBER_CLASSES, activation='sigmoid'))
    model.compile(optimizer='adam', loss=weighted_cross_entropy, metrics=[AUC(multi_label=True)])
    return model

In [29]:
NUMBER_OF_IMAGES = 10000
IMAGE_SIZE = 244
BATCH_SIZE = 16

def normalize_image(img, label):
    img = tf.cast(img, np.float32) / 255.0
    return img, label

images, labels = prepare_data(df[:NUMBER_OF_IMAGES], IMAGE_SIZE)
images_train, images_val, labels_train, labels_val = sklearn.model_selection.train_test_split(images, labels, random_state=42, test_size=0.2)
images_test, labels_test = prepare_data(df[-(int(NUMBER_OF_IMAGES / 5)):], IMAGE_SIZE)

train_dataset = tf.data.Dataset.from_tensor_slices((images_train, labels_train))
train_dataset = train_dataset.map(normalize_image)
train_dataset = train_dataset.shuffle(buffer_size=100)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
train_dataset = train_dataset.repeat()

val_dataset = tf.data.Dataset.from_tensor_slices((images_val, labels_val))
val_dataset = val_dataset.map(normalize_image)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=False)

model = create_resnet_model(IMAGE_SIZE)

100%|██████████| 2000/2000 [00:19<00:00, 104.22it/s]


In [35]:
steps_per_epoch = len(images_train) // BATCH_SIZE
validation_steps = len(images_val) // BATCH_SIZE

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC(multi_label=True)])

model.fit(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=val_dataset,
    validation_steps=validation_steps,
)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 79s 145ms/step - auc_19: 0.6488 - loss: 0.1987 - val_auc_19: 0.6187 - val_loss: 0.2000
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 64s 129ms/step - auc_19: 0.6583 - loss: 0.1983 - val_auc_19: 0.6250 - val_loss: 0.2115
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 64s 128ms/step - auc_19: 0.6582 - loss: 0.1989 - val_auc_19: 0.6237 - val_loss: 0.1965
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 64s 128ms/step - auc_19: 0.6551 - loss: 0.1982 - val_auc_19: 0.6182 - val_loss: 0.1974
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 64s 128ms/step - auc_19: 0.6647 - loss: 0.1971 - val_auc_19: 0.6194 - val_loss: 0.1991
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 64s 128ms/step - auc_19: 0.6645 - loss: 0.1978 - val_auc_19: 0.6281 - val_loss: 0.1981
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 64s 127ms/step - auc_19: 0.6588 - loss: 0.1969 - val_auc_19: 0.6244 - val_loss: 0.1975
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 64s 127ms/step - auc_19: 0.6628 - loss: 0.1971 - val_auc_19: 0.6215 - v

In [39]:
def evaluate_model(model, images_test, labels_test, batch_size=BATCH_SIZE):

    images_test = tf.cast(images_test, np.float32) / 255.0
    
    predictions = model.predict(images_test, batch_size=batch_size)
    
    auc_per_class = []
    
    for class_idx in range(NUMBER_CLASSES):
        true_labels_class = labels_test[:, class_idx]
        
        auc = sklearn.metrics.roc_auc_score(true_labels_class, predictions[:, class_idx])
        auc_per_class.append(auc)
    
    return auc_per_class

print(evaluate_model(model, images_test, labels_test))

def add_no_finding_class(labels):
    updated_labels = np.hstack([labels, np.zeros((labels.shape[0], 1))])  # Add a column of zeros for the 'No Finding' class
    updated_labels[np.sum(labels, axis=1) == 0, -1] = 1
    return updated_labels



125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 92ms/step
[0.6565615502255261, 0.5994708618914024, 0.6547018374226965, 0.4897615438292548, 0.6740580158001882, 0.5003462480911964, 0.5879024144869216, 0.8531130028375897, 0.6064896289529541, 0.5312953025254965, 0.4909565388655462, 0.4100576681511934, 0.42556073588709675, 0.569589093701997, 0.6177235733069157]


All labels: 10000 samples, 50 epochs: 
[0.6692994173103355, 0.5939930395255514, 0.6683094818155433, 0.5533819389301956, 0.6756526600600586, 0.5141695372704997, 0.5874413145539906, 0.814388249040227, 0.6113292270077111, 0.5362483786464559, 0.5052575717787114, 0.39618781165543754, 0.4708606350806452, 0.5637999231950844] (No Finding missing)

10 epochs:
[0.6565615502255261, 0.5994708618914024, 0.6547018374226965, 0.4897615438292548, 0.6740580158001882, 0.5003462480911964, 0.5879024144869216, 0.8531130028375897, 0.6064896289529541, 0.5312953025254965, 0.4909565388655462, 0.4100576681511934, 0.42556073588709675, 0.569589093701997, 0.6177235733069157]


In [38]:
ALL_LABELS


['Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'Effusion',
 'Emphysema',
 'Fibrosis',
 'Hernia',
 'Infiltration',
 'Mass',
 'No Finding',
 'Nodule',
 'Pleural_Thickening',
 'Pneumonia',
 'Pneumothorax']